### Importing necessary library

In [ ]:
import pandas as pd
import numpy as np

import datetime

### Reading the data file

In [ ]:
rent_df = pd.read_csv('../Data/canada_rent.csv')
rent_df

### Investigate the possibility of each categorical columns

In [ ]:

print('The options for lease_term are : ', rent_df['lease_term'].unique(), '\n')

print('The options for type are : ', rent_df['type'].unique(), '\n')

print('The options for furnishing are : ', rent_df['furnishing'].unique(), '\n')

print('The options for smoking are : ', rent_df['smoking'].unique(), '\n')

print('The options for cats are : ', rent_df['cats'].unique(), '\n')

print('The options for dogs are : ', rent_df['dogs'].unique(), '\n')

print('The options for beds are : ', rent_df['beds'].unique(), '\n')

In [ ]:
rent_df['furnishing'].replace({'Unfurnished, Negotiable': 'Negotiable'}, inplace=True)

### Cleaning up the dataframe

I will be removing columns that will bring no additional value to the regression analysis
  * such as link and rentfaster_id columns.

I will also be working with the latitude and longitude instead of address, since converting the addresses to 1/0 columns would be massive.
  * I will be removing the address column.
  * However, I will keep the city and Province for now.

I will be removing the string 'Beds' from the beds column and will be replacing studio and none Beds by 0.

I will be replacing the string immediate in the column availability_date by today's date, in order to change the column to a datetime stamp.

I will be dropping all the rows where the column availability_date has the string 'No Vacancy'.

I will be replacing the string 'Negotiable' and 'Call for Availability' in the column availability_date by today's date.

I will be combining similar terms in the furnishing column.  'Unfurnished, Negotiable' will become 'Negotiable'.

I will be replacingsimilar terms in the smoking column. 'Smoke Free Building' replaced by 'Non-Smoking'.

In [ ]:
# Removing unecessary columns
rent_df.drop(columns=['link', 'rentfaster_id','address'], inplace=True)

**Filtering the beds column**

In [ ]:
# Looking at the beds possibilities
print('Before =>',rent_df['beds'].unique())

# Removing Beds, Bed from column beds
rent_df['beds'].replace(r'\s*Beds?\s*','', regex=True, inplace=True)

# Replacing Studio and none by 0
rent_df['beds'].replace(r'\b(Studio|none)\b','0', regex=True, inplace=True)

# Verifying that the string replacement was done correctly
print('After =>',rent_df['beds'].unique())

**Filtering the availability_date column**

In [ ]:
# Look the possible string the column availability_date
print('Before =>', rent_df['availability_date'].unique())

# Looking at frequency of occurency of 'Negotiable', 'Call for Availability' in the column availability_date
print(rent_df[rent_df['availability_date'].isin(['Negotiable', 'Call for Availability'])].count())

# Looking at frequency of occurency of 'No Vacancy' in the column availability_dateabs
print(rent_df[rent_df['availability_date'].isin(['No Vacancy'])].count())


In [ ]:
# Removing adds (removing the entire row) where there is 'No Vacancy'
rent_df = rent_df[~rent_df['availability_date'].isin(['No Vacancy'])]

In [ ]:
# Retrieve today's date
today = datetime.date.today().strftime('%B %d') #pd.Timestamp.today()

# Replace the strings by today's date
rent_df['availability_date'].replace({'Immediate': today}, inplace=True)
rent_df['availability_date'].replace({'Negotiable': today}, inplace=True)
rent_df['availability_date'].replace({'Call for Availability': today}, inplace=True)

print('After =>', rent_df['availability_date'].unique())

# Convert the column availability_date to a timestamp
rent_df['availability_date'] = pd.to_datetime(rent_df['availability_date'] + f", {datetime.date.today().year}", format="%B %d, %Y")

rent_df['availability_date']

**Combining similar string in the furnishing column**

In [ ]:
# Look the possible string the column furnishing
print('Before =>', rent_df['furnishing'].unique())

rent_df['furnishing'].replace({'Unfurnished, Negotiable': 'Negotiable'}, inplace=True)

# Look the possible string the column furnishing
print('After =>', rent_df['furnishing'].unique())

**Combining similar string in the smoking column**

In [ ]:
# Look the possible string the column smoking
print('Before =>', rent_df['smoking'].unique())

# Replace 'Smoke Free Building' by 'Non-Smoking'
rent_df['smoking'].replace({'Smoke Free Building': 'Non-Smoking'}, inplace=True)

# Replace 'Negotiable' by 'Smoking Allowed'
rent_df['smoking'].replace({'Negotiable': 'Smoking Allowed'}, inplace=True)

# Look the possible string the column smoking
print('After =>', rent_df['smoking'].unique())

**Cleaning up the sq_feet column**

In [ ]:
# Removing the sq ft unit sometimes mentioned
rent_df['sq_feet'] = rent_df['sq_feet'].str.split(' ', expand=True)[0]

# Replace + sign
rent_df['sq_feet'] = rent_df['sq_feet'].str.split('+', expand=True)[0]

# Replace , 
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace(',', '', regex=True)


# Removing range.
rent_df['sq_feet'] = rent_df['sq_feet'].str.split('-', expand=True)[0]

# Removing range.
rent_df['sq_feet'] = rent_df['sq_feet'].str.split('.', expand=True)[0]

# Replace , 
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('~', '', regex=True)

# Replace , 
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('sf', '', regex=True)

# Replace , 
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('Large', '', regex=True)

# Replace , 
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('Large', '', regex=True)

# Replace 'approx.' 
rent_df['sq_feet'] = rent_df['sq_feet'].replace(r'approx', np.nan, regex=True)
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('Approximately.', '', regex=True)
rent_df['sq_feet'] = rent_df['sq_feet'].str.replace('Approximately', '', regex=True)

# Replace 'approx.' 
rent_df['sq_feet'] = rent_df['sq_feet'].replace(r'TOL', np.nan, regex=True)

rent_df['sq_feet'] = rent_df['sq_feet'].str.replace(r'[^\d]', '', regex=True)

# Replace '' 
rent_df['sq_feet'] = rent_df['sq_feet'].replace('', np.nan, regex=True)

rent_df['sq_feet'] = pd.to_numeric(rent_df['sq_feet'], errors='coerce')

### Investigate if the dataset contains missing values

In [ ]:
rent_df.isna().sum()

I want to dig into the missing value.

I will look for what type of property has no square footage indicated. I might want to drop certain property type (ie: storage, garage, ...)

In [ ]:
(rent_df[rent_df['sq_feet'].isna()].groupby(['type'])['city'].count() / rent_df.groupby(['type'])['city'].count()) *100

I will be removing all the storage, since they all don't include square footage.

I will also be removing the Parking spot category, since most of them are missing the square footage.

I will also be removing the Office Space and Acreage to focus mainly on standard housing


In [ ]:
# Removing the Property type based on the previous analysis
rent_df = rent_df[~rent_df['type'].isin(['Acreage', 'Office Space', 'Parking Spot', 'Storage'])]

In [ ]:
rent_df.isna().sum()

In [ ]:
(rent_df[rent_df['sq_feet'].isna()].groupby(['type'])['city'].count() / rent_df.groupby(['type'])['city'].count()) *100

Replacing the the Nan value in sq_feet by the grouped mean on building type and # of beds
  * Even after replacing the Nan value there are some group categories that just don't have non nan data to produce a mean.
  * I will be dropping two remaining rows of nan value

In [ ]:
# Replacing Nan
rent_df['sq_feet'] = rent_df.groupby(['type', 'beds'])['sq_feet'].transform(lambda x: x.fillna(x.mean()))

# Verifying that there is no longer nan value for the sq_feet
rent_df.isna().sum()

# Investigating the remaining Nan
group_means = rent_df.groupby(['type', 'beds'])['sq_feet'].mean()

print(group_means[group_means.isna()])

print(rent_df[rent_df['type'] == 'House'].groupby('beds')['sq_feet'].mean())

# Dropping remaininf Nan in sq_feet
rent_df = rent_df[~rent_df['sq_feet'].isna()]

# Converting the column to int
rent_df['sq_feet'] = rent_df['sq_feet'].astype(int)

# Verifying that there is no longer nan value for the sq_feet
rent_df.isna().sum()

I will be dropping the Nan value of lease_term because it implies only a small amount of data on the entire dataset

In [ ]:
# Dropping remaininf Nan in sq_feet
rent_df = rent_df[~rent_df['lease_term'].isna()]

# Verifying that there is no longer nan value for the sq_feet
rent_df.isna().sum()

I will be replacing Nan value in smoking according to the most frequent smoking category, which is Non-Smocking.

In [ ]:
# Look at the occurence count of each smoking category
print(rent_df.groupby(['smoking'])['city'].count())

# Replace Nan value in smoking
rent_df['smoking'] = rent_df['smoking'].fillna('Non-Smoking')

# Verifying that there is no longer nan value for the smoking
rent_df.isna().sum()

I will be replacing Nan value in availability_date by today's date 

In [ ]:
rent_df['availability_date'] = rent_df['availability_date'].fillna(f"{datetime.date.today().strftime('%B %d')}, {datetime.date.today().year}")

# Verifying that there is no longer nan value for the smoking
rent_df.isna().sum()

### Saving the cleaned data to a csv file

In [ ]:
# Reset index
rent_df.reset_index(drop=True)

# Writing to file 
rent_df.to_csv('../Data/canada_rent_clean.csv', index=False)
